# Experiment: Model Sensitivity to Imbalanced Data (Geographic Regions)

## Objective
Investigate how geographic imbalance in training data affects model performance on underrepresented regions and explore mitigation through oversampling.

## Data Analysis:

Analyze the distribution of data across continents using departure_continent.

Identify underrepresented regions (e.g., Africa, South America) compared to overrepresented ones (e.g., Europe, North America).

Data Splitting:

In [6]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import joblib
from smogn import smoter

In [3]:
# Load data
X_train = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/X_train.parquet")
X_test = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/X_test.parquet")
X_val = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/X_val.parquet")
y_train = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/y_train.parquet").squeeze()
y_test = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/y_test.parquet").squeeze()
y_val = pd.read_parquet("https://github.com/monatagelsir7/enivornmental_impact_of_aviation/raw/refs/heads/main/Test-Train-Validation%20Data/y_val.parquet").squeeze()

In [7]:
continent_cols = [c for c in X_train.columns if "departure_continent_" in c]
continent_dist = X_train[continent_cols].sum().sort_values(ascending=False)
print("Training Data Continent Distribution:\n", continent_dist / len(X_train))

Training Data Continent Distribution:
 departure_continent_NA    0.441307
departure_continent_EU    0.367883
departure_continent_AS    0.131445
departure_continent_SA    0.020127
departure_continent_OC    0.010175
dtype: float64


In [9]:
# 2. Artificially imbalance: Remove South America (SA) and Oceania (OC)
imbalanced_train = X_train[
    (X_train["departure_continent_SA"] == 0) & 
    (X_train["departure_continent_OC"] == 0)
]

In [10]:
# 3. Train baseline model on imbalanced data
rf = RandomForestRegressor(n_estimators=61, random_state=42, n_jobs=-1)
rf.fit(imbalanced_train, y_train.loc[imbalanced_train.index])


RandomForestRegressor(n_estimators=61, n_jobs=-1, random_state=42)

In [15]:
# 4. Oversample minority regions (SA/OC) using SMOGN
imbalanced_train = imbalanced_train.reset_index(drop=True)
y_train_subset = y_train.loc[imbalanced_train.index].reset_index(drop=True)

# Combine features and target
train_data = pd.concat([imbalanced_train, y_train_subset], axis=1)

# Oversample using SMOGN
balanced_data = smoter(
    data=train_data,
    y='co2_per_distance',
    k=5,
    pert=0.1
)



KeyError: '[0, 1, 4, 6, 10, 11, 12, 14, 16, 20, 21, 22, 24, 26, 27, 33, 34, 35, 41, 42, 44, 49, 50, 53, 57, 60, 61, 70, 79, 80, 81, 87, 88, 89, 91, 93, 95, 96, 99, 101, 105, 106, 107, 110, 112, 114, 117, 118, 119, 120, 123, 124, 128, 129, 130, 131, 142, 144, 148, 150, 151, 155, 156, 163, 164, 166, 170, 172, 174, 176, 178, 183, 184, 186, 188, 193, 195, 204, 207, 208, 210, 211, 214, 220, 221, 222, 227, 233, 237, 238, 242, 244, 247, 254, 256, 260, 262, 268, 271, 278, 283, 284, 285, 286, 287, 288, 289, 294, 296, 297, 298, 302, 304, 305, 307, 310, 321, 323, 328, 329, 333, 338, 351, 353, 354, 355, 359, 361, 362, 369, 370, 371, 375, 376, 377, 382, 386, 387, 391, 396, 397, 400, 401, 406, 415, 419, 422, 427, 435, 436, 437, 438, 445, 452, 458, 459, 460, 464, 465, 468, 472, 480, 483, 491, 494, 495, 499, 500, 502, 505, 506, 511, 512, 513, 514, 517, 521, 522, 523, 530, 544, 549, 550, 556, 567, 569, 570, 577, 578, 580, 583, 589, 590, 595, 596, 597, 602, 607, 612, 628, 629, 631, 632, 634, 635, 639, 6

In [ ]:
# 5. Retrain on balanced data
rf_balanced = RandomForestRegressor(n_estimators=61, random_state=42, n_jobs=-1)
rf_balanced.fit(balanced_data.drop('co2', axis=1), balanced_data['co2'])

In [ ]:
# 7. Evaluate performance
def evaluate_model(model, name):
    y_pred = model.predict(X_test)
    print(f"\n{name} Model Performance:")
    print(f"Global R²: {r2_score(y_test, y_pred):.3f}")
    
    # Evaluate on underrepresented regions
    for continent in ['SA', 'OC']:
        mask = X_test[f"departure_continent_{continent}"] == 1
        if mask.sum() > 0:
            y_pred_region = y_pred[mask]
            y_true_region = y_test[mask]
            print(f"{continent} - R²: {r2_score(y_true_region, y_pred_region):.3f}")
    
evaluate_model(rf, "Imbalanced")
evaluate_model(rf_balanced, "Balanced")